In [1]:
from sage.all import *
from sage.modules.all import *
import random

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Required for 3D plotting
from collections import Counter
import plotly.graph_objects as go
import plotly.offline as pyo  # Import plotly.offline for controlling osutput
from collections import defaultdict
import json

import csv
import json
import numpy as np

import funcs
from importlib import reload

reload(funcs)
from funcs import *

Parallelism().set(nproc=8)

def convert(o):
    if isinstance(o, (Integer,)):
        return int(o)
    if isinstance(o, (np.floating,)):
        return float(o)
    if isinstance(o, (np.ndarray,)):
        return o.tolist()
    if isinstance(o, (set,)):
        return list(o)
    raise TypeError(f"Object of type {o.__class__.__name__} is not JSON serializable")

r = 2
m = 7
n = 8
left = 4
right = 15
step = 1
expc = 10

len_data = (right - left) * expc

In [21]:
def find_hull_dims(r, m, noisedcounts, exp = 10, cores = 8):
    c = codes.BinaryReedMullerCode(r, m)
    
    @parallel(ncpus=cores)
    def func(count):
        (genmatr, noisedcoords) = apply_matrix_noise(c.generator_matrix(), count)
        hull = code_hull(Matrix(GF(2), genmatr))
        dims = hull.dimension()
        return genmatr, hull, dims
        
    res = list()
    inps = [j for j in noisedcounts for i in range(exp)]
    for inp, out in func(inps):
        res.append((inp[0][0], out))
        
    return res

In [22]:
hull_data = find_hull_dims(r, m, range(0, 20, 1), 100)

In [ ]:
hull_dims = [ (hull_data[i][0], hull_data[i][1][2]) for i in range(len(hull_data))]

with open("hull_dims.json", "w") as f:
    json.dump(hull_dims, f)

print(hull_dims)

freq_map = defaultdict(lambda: defaultdict(int))

for key, value in hull_dims:
    freq_map[key][value] += 1

hull_dims_freq = {k: dict(v) for k, v in freq_map.items()}

print(hull_dims_freq)

with open("hull_dims_freq.json", "w") as f:
    json.dump(hull_dims_freq, f)

[(0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (0, 29), (1, 27), (1, 27), (1, 27), (1, 27), (1, 27), (1, 27), (1, 27), (1, 27), (1, 27), (1, 27), (1, 27), 

In [141]:
for key in hull_dims_freq:
    print(key, ":", hull_dims_freq[key], end='\n')

0 : {29: 100}
1 : {27: 100}
2 : {25: 100}
3 : {23: 100}
4 : {21: 100}
5 : {19: 100}
6 : {17: 100}
7 : {15: 100}
8 : {13: 100}
9 : {11: 100}
10 : {9: 100}
11 : {7: 100}
12 : {5: 97, 6: 2, 7: 1}
13 : {3: 86, 4: 8, 5: 6}
14 : {1: 64, 2: 20, 3: 15, 4: 1}
15 : {1: 48, 2: 20, 3: 5, 0: 27}
16 : {0: 35, 1: 46, 2: 17, 3: 1, 4: 1}
17 : {1: 36, 0: 41, 2: 21, 3: 2}
18 : {0: 39, 1: 49, 2: 9, 3: 3}
19 : {1: 47, 2: 13, 3: 2, 0: 38}


In [3]:
data = find_coords(r, m, range(left, right, step), expc)

Overall Progress: 100%|██████████| 330/330 [1:34:29<00:00, 17.18s/it]  


In [4]:
find_coords_res_dump = []

i = 0
for inp, out in data:
    find_coords_res_dump.append([inp[0], i, list(out[0][1]), list(out[0][2])])
    save(out[0][0], f"matrices2/{i}.sobj")
    i += 1

with open("find_coords_res_dump.json", "w") as f:
    json.dump(find_coords_res_dump, f)
    
find_coords_res = []
for inp, out in data:
    find_coords_res.append([inp[0], out[0][0], out[0][1], out[0][2]])

In [2]:
find_coords_res = []

with open("find_coords_res_dump.json", "r") as f:
    find_coords_res_dump = json.load(f)

for i in range(len(find_coords_res_dump)):
    find_coords_res.append([find_coords_res_dump[i][0], load(f"matrices2/{find_coords_res_dump[i][1]}.sobj"), set(find_coords_res_dump[i][2]), set(find_coords_res_dump[i][3])])
    
print(find_coords_res)

[[4, [256, 37] linear code over GF(2), {217, 130, 188, 231}, {217, 130, 188, 231}], [4, [256, 37] linear code over GF(2), {151, 237, 86, 143}, {143, 237, 86, 151}], [4, [256, 37] linear code over GF(2), {88, 33, 86, 247}, {88, 33, 86, 247}], [4, [256, 37] linear code over GF(2), {187, 252, 229, 239}, {187, 252, 229, 239}], [4, [256, 37] linear code over GF(2), {194, 140, 36, 23}, {194, 140, 36, 23}], [4, [256, 37] linear code over GF(2), {160, 65, 220, 249}, {160, 249, 220, 65}], [4, [256, 37] linear code over GF(2), {168, 154, 207, 31}, {168, 154, 31, 207}], [4, [256, 37] linear code over GF(2), {248, 233, 58, 69}, {248, 233, 58, 69}], [4, [256, 37] linear code over GF(2), {250, 99, 5, 94}, {250, 99, 5, 94}], [4, [256, 37] linear code over GF(2), {169, 27, 233, 119}, {169, 27, 233, 119}], [4, [256, 37] linear code over GF(2), {8, 129, 72, 62}, {8, 129, 72, 62}], [4, [256, 37] linear code over GF(2), {128, 73, 30, 97}, {128, 73, 30, 97}], [4, [256, 37] linear code over GF(2), {216, 113

In [3]:
freq_map = defaultdict(lambda: defaultdict(int))

for x in find_coords_res:
    freq_map[x[0]][len(x[2] & x[3]) / max(len(x[3]), 1)] += 1
    
find_coords_freq = {k: dict(v) for k, v in freq_map.items()}

with open("find_coords_freq.json", "w") as f:
    json.dump(find_coords_freq, f)

In [4]:
for key in find_coords_freq:
    print(key, ":", find_coords_freq[key], end='\n')

4 : {1.0: 30}
5 : {1.0: 30}
6 : {1.0: 30}
7 : {1.0: 30}
8 : {1.0: 30}
9 : {1.0: 30}
10 : {1.0: 30}
11 : {1.0: 30}
12 : {1.0: 30}
13 : {1.0: 30}
14 : {1.0: 30}


In [5]:
freq_map = defaultdict(lambda: defaultdict(int))

for x in find_coords_res:
    freq_map[x[0]][len(x[2]) / float(max(len(x[3]), 1))] += 1
    
find_coords_freq_2 = {k: dict(v) for k, v in freq_map.items()}

for key in find_coords_freq_2:
    print(key, ":", find_coords_freq_2[key], end='\n')

# codes = [data[i][1][0][0] for i in range(len(data))]
# coords = [data[i][1][0][2] for i in range(len(data))]
# get_noised_data = get_noised(codes, [10 for i in range(len(codes))], coords)


4 : {1.0: 30}
5 : {1.0: 30}
6 : {1.0: 30}
7 : {1.0: 30}
8 : {1.0: 30}
9 : {1.0: 30}
10 : {1.0: 30}
11 : {1.0: 30}
12 : {1.0: 29, 1.0833333333333333: 1}
13 : {1.0: 30}
14 : {1.0: 26, 1.0714285714285714: 2, 1.1428571428571428: 2}


In [6]:
noisedcodes = [load(f'matrices2/{j}.sobj') for j in range(len_data)]

In [7]:
err_counts = [find_coords_res[j][0] for j in range(len_data)]
coordss = [find_coords_res[j][2] for j in range(len_data)]

In [8]:
results = get_noised(noisedcodes, err_counts, coordss)

Overall Progress: 100%|██████████| 330/330 [00:06<00:00, 53.26it/s] 


In [9]:
for k in results:
    print(k, ":", results[k])

5 : {160, 65, 220, 249}
6 : {168, 154, 31, 207}
7 : {248, 233, 58, 69}
8 : {250, 99, 5, 94}
9 : {169, 27, 233, 119}
10 : {8, 129, 72, 62}
11 : {128, 73, 30, 97}
12 : {216, 113, 116, 165}
13 : {216, 65, 47, 199}
14 : {217, 187, 188, 93}
15 : {128, 10, 46, 154}
16 : {65, 11, 229, 35}
17 : {224, 145, 134, 158}
18 : {169, 114, 117, 207}
19 : {128, 121, 181, 158}
20 : {89, 154, 177, 85}
21 : {74, 35, 133, 39}
22 : {80, 227, 222, 14}
23 : {16, 203, 125, 119}
24 : {80, 57, 51, 6}
25 : {24, 106, 100, 159}
26 : {121, 75, 149, 253}
4 : {194, 140, 36, 23}
28 : {228, 135, 72, 178, 216}
29 : {184, 121, 198, 46}
30 : {101, 184, 57, 62, 191}
31 : {137, 242, 149, 127}
32 : {235, 242, 185, 155, 252}
33 : {34, 42, 76, 109, 144}
34 : {9, 175, 117, 218, 221}
35 : {129, 48, 241, 244, 119}
36 : {201, 84, 151, 26, 61}
37 : {225, 232, 234, 47, 157}
38 : {192, 35, 235, 240, 19}
39 : {163, 230, 179, 182, 95}
27 : {112, 75, 108, 236}
41 : {161, 169, 170, 43, 237}
42 : {193, 168, 80, 149, 29}
43 : {73, 52, 148, 1

In [10]:
results_dump = [list( (k, list(results[k])) ) for k in results]

with open("get_noised_res.json", "w") as f:
    json.dump(results_dump, f)

In [11]:
results_copy = results.copy()

In [12]:
freq_map = defaultdict(lambda: defaultdict(int))

for i in range(len(results_copy)):
    freq_map[int(i / expc + left)][len(results_copy[i] & find_coords_res[i][3]) / max(len(find_coords_res[i][3]), 1)] += 1
    
get_noised_freq = {k: dict(v) for k, v in freq_map.items()}

In [13]:
for key in get_noised_freq:
    print(key, ":", get_noised_freq[key], end='\n')

4 : {1.0: 30}
5 : {1.0: 30}
6 : {1.0: 30}
7 : {1.0: 30}
8 : {1.0: 30}
9 : {1.0: 30}
10 : {1.0: 30}
11 : {1.0: 30}
12 : {1.0: 30}
13 : {1.0: 30}
14 : {1.0: 30}


In [31]:
dist_res = distillation(noisedcodes, err_counts, coordss)

Overall Progress: 100%|██████████| 330/330 [00:01<00:00, 210.86it/s]


# NEW ALGOs

In [120]:
data = find_noised_candidates(r, m, range(left, right, step), expc)

Overall Progress: 100%|██████████| 110/110 [06:03<00:00,  3.30s/it]


In [124]:
find_noised_candidates_res_dump = []

i = 0
for inp, out in data:
    find_noised_candidates_res_dump.append([int(inp[0]), int(i), list(out[0][1]), list(out[0][2])])
    save(out[0][0], f"matrices4/{i}.sobj")
    i += 1

with open("find_noised_candidates_res_dump.json", "w") as f:
    json.dump(find_noised_candidates_res_dump, f)

find_noised_candidates_res = []
with open("find_noised_candidates_res_dump.json", "r") as f:
    find_noised_candidates_res_dump = json.load(f)

for i in range(len(find_noised_candidates_res_dump)):
    find_noised_candidates_res.append([int(find_noised_candidates_res_dump[i][0]), load(f"matrices4/{find_noised_candidates_res_dump[i][1]}.sobj"), set(find_noised_candidates_res_dump[i][2]), set(find_noised_candidates_res_dump[i][3])])

In [127]:
freq_map = defaultdict(lambda: defaultdict(int))

for x in find_noised_candidates_res:
    freq_map[x[0]][len(x[2]) / max(len(x[3]), 1)] += 1

find_noised_candidates_res_freq = {k: dict(v) for k, v in freq_map.items()}

with open("find_noised_candidates_res_freq.json", "w") as f:
    json.dump(find_noised_candidates_res_freq, f, default=convert)

In [128]:
for key in find_noised_candidates_res_freq:
    print(key, ":", find_noised_candidates_res_freq[key], end='\n')

4 : {1.0: 10}
5 : {1.0: 10}
6 : {1.0: 10}
7 : {1.0: 10}
8 : {1.0: 10}
9 : {1.0: 10}
10 : {1.0: 6, 1.2: 1, 1.1: 3}
11 : {1.1818181818181819: 1, 1.0909090909090908: 5, 1.0: 4}
12 : {1.1666666666666667: 2, 1.3333333333333333: 2, 1.0: 1, 1.4166666666666667: 1, 1.75: 1, 1.5833333333333333: 1, 1.0833333333333333: 1, 1.5: 1}
13 : {2.4615384615384617: 5, 2.3076923076923075: 2, 1.6923076923076923: 1, 2.1538461538461537: 1, 2.0: 1}
14 : {4.571428571428571: 1, 5.142857142857143: 2, 9.142857142857142: 7}


In [109]:
noisedcodes = [load(f'matrices4/{j}.sobj') for j in range(len_data)]
err_counts = [find_noised_candidates_res[j][0] for j in range(len_data)]
coordss = [find_noised_candidates_res[j][2] for j in range(len_data)]

## Get Noised

In [110]:
get_noised_res = get_noised(noisedcodes, err_counts, coordss)

Overall Progress: 100%|██████████| 110/110 [06:04<00:00,  3.32s/it]


In [111]:
get_noised_res_dump = [list( (k, list(get_noised_res[k])) ) for k in get_noised_res]

with open("get_noised_res.json", "w") as f:
    json.dump(get_noised_res_dump, f, default=convert)

In [112]:
freq_map = defaultdict(lambda: defaultdict(int))

for i in range(len(get_noised_res)):
    freq_map[int(i / expc + left)][len(get_noised_res[i] & find_noised_candidates_res[i][3]) / max(len(find_noised_candidates_res[i][3]), 1)] += 1

get_noised_freq = {k: dict(v) for k, v in freq_map.items()}

In [113]:
for key in get_noised_freq:
    print(key, ":", get_noised_freq[key], end='\n')

4 : {1.0: 10}
5 : {1.0: 10}
6 : {1.0: 10}
7 : {1.0: 10}
8 : {1.0: 10}
9 : {1.0: 10}
10 : {1.0: 10}
11 : {1.0: 10}
12 : {1.0: 9, 0.8333333333333334: 1}
13 : {0.0: 2, 0.07692307692307693: 2, 1.0: 2, 0.21428571428571427: 1, 0.2857142857142857: 1, 0.23076923076923078: 1, 0.6153846153846154: 1}
14 : {0.46153846153846156: 1, 0.0: 2, 0.07142857142857142: 3, 0.2857142857142857: 2, 0.14285714285714285: 2}


## Distillation

In [114]:
distillation_res = distillation(noisedcodes, err_counts, coordss)

Overall Progress: 100%|██████████| 110/110 [13:42<00:00,  7.48s/it]


In [115]:
distillation_res_dump = [list( (k, list(distillation_res[k])) ) for k in distillation_res]

with open("distillation_res.json", "w") as f:
    json.dump(distillation_res, f, default=convert)

In [116]:
freq_map = defaultdict(lambda: defaultdict(int))

for i in range(len(distillation_res)):
    freq_map[int(i / expc + left)][len(set(distillation_res[i]) & find_noised_candidates_res[i][3]) / max(len(find_noised_candidates_res[i][3]), 1)] += 1

distillation_freq = {k: dict(v) for k, v in freq_map.items()}

In [117]:
for key in distillation_freq:
    print(key, ":", distillation_freq[key], end='\n')

4 : {1.0: 10}
5 : {1.0: 10}
6 : {1.0: 10}
7 : {1.0: 10}
8 : {1.0: 10}
9 : {1.0: 10}
10 : {1.0: 10}
11 : {1.0: 10}
12 : {1.0: 9, 0.9166666666666666: 1}
13 : {0.9230769230769231: 3, 1.0: 7}
14 : {1.0: 5, 0.7142857142857143: 1, 0.21428571428571427: 1, 0.9285714285714286: 1, 0.8571428571428571: 1, 0.6428571428571429: 1}


In [9]:
# запустите в sage (sage -python) или в ноутбуке sagecell
from sage.all import *
# В библиотеке авторов реализованы конструкции; пробуйте стандартный интерфейс:
H = hadamard_matrix(1604)   # если интерфейс есть — вернёт матрицу +1/-1
# или, если authors' module предоставляет named constructor:
# from sage.combinat.matrices.hadamard_matrix import hadamard_matrix
# H = hadamard_matrix(1608)
# Сохраним в файл (CSV или numpy)
import numpy as np
A = np.array([list(map(int,row)) for row in H])   # приведём к ndarray
with open("Hadamard_matrix.txt", "w") as f:
    print(H, file=f)